# Yann Cauchepin

Hi, here is my documented jupyter notebook which responds to the hands-on test. I tried to make it fast to compute to save your time.

# Requirements

Before even running the following script, please follow the first steps:

- [ ] Installing the necessary librairies. You can comment the next script to avoid biding your time.

- [ ] Replacing dataset path toward your correct local repositories.

In [1]:
!pip install scipy
!pip install tqdm
!pip install pandas
!pip install numpy
!pip install pyspark
!pip install catboost
!pip install shap
!pip install pingouin
!pip install seaborn
!pip install torch
!pip install captum
!pip install sklearn
!pip install mapie
!pip install scikit-optimize
!pip install pickle

Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
^C
ERROR: Operation cancelled by user
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package 

In [2]:
metadata_path = "/media/yanncauchepin/ExternalDisk/Datasets/MachineLearningTables/lending_club/LCDataDictionary.xlsx"
data_path = "/media/yanncauchepin/ExternalDisk/Datasets/MachineLearningTables/lending_club/Loan_status_2007-2020Q3.csv"

# Data Importation

I initially plan to use **Pandas** librairy as I am used to work on few data, either in my current job or in my hands-on exercices. Since my computer freeze over many times, I decided to use **[Spark](https://spark.apache.org/)** library to manipulate this significative amount of data. This library involve map-reduce method to handle big data more efficiently.

In [3]:
import pandas as pd
import numpy as np

In [4]:
metadata = pd.read_excel(metadata_path, index_col=0)
metadata = metadata.iloc[:-2,:]

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LendingClubDataProcessing") \
    .getOrCreate()

df_spark = spark.read.csv(data_path, header=True, inferSchema=True)
print(f"Number of data: {df_spark.count()}")

24/06/23 14:57:44 WARN Utils: Your hostname, yanncauchepincomputer resolves to a loopback address: 127.0.1.1; using 192.168.43.208 instead (on interface wlp2s0)
24/06/23 14:57:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 14:57:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/23 14:57:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/yanncauchepin/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=57>

During handling of the above exception, another exception occurred

# Data Cleaning - Encoding Target

Since the dataset was filled with metadata, I decided to check the consistency of the data to remove outer features. Additionnaly, I follow the instruction of removing *grade* and *sub_grade* features.

Since I understood that the machine learning must classify whether a loan is *Fully Paid* or *Charged Off* specifically, I decided to encode the rest to NaN.

In [ ]:
print(f"Metadata features: {len(metadata.index)}")
print(f"Data features: {len(df_spark.columns)}")

outer_features = [feature for feature in df_spark.columns if feature not in metadata.index]

print(f"Unknown data features: {outer_features} ({len(outer_features)})")
df_spark = df_spark.drop(*outer_features)

In [ ]:
features_to_drop = ['grade', 'sub_grade']
df_spark = df_spark.drop(*features_to_drop)

In [ ]:
value_counts = df_spark.groupBy('loan_status').count()
value_rate = value_counts.count() / df_spark.count()
print(f"Number of distincts values: {value_counts.count()} - {value_rate:.2e} %")
value_counts.show()

In [ ]:
'''
mapping = {
    'Fully Paid': 0,
    'Charged Off': 1,
    'Current': np.nan,
    'Late (31-120 days)': np.nan,
    'In Grace Period': np.nan,
    'Late (16-30 days)': np.nan,
    'Issued': np.nan,
    'Does not meet the credit policy. Status:Fully Paid': np.nan,
    'Does not meet the credit policy. Status:Charged Off': np.nan,
    'Default': np.nan,
    'Oct-2015': np.nan
}
'''

from pyspark.sql.functions import when
df_spark = df_spark.withColumn("loan_status", when(df_spark["loan_status"] == "Fully Paid", 0)
                   .when(df_spark["loan_status"] == "Charged Off", 1)
                   .otherwise(np.nan))

In [ ]:
all_features = [feature for feature in df_spark.columns if feature != 'loan_status']
df_spark = df_spark.fillna({feature: "nan" if df_spark.schema[feature].dataType == 'string' else np.nan for feature in all_features})

In [ ]:
value_counts = df_spark.groupBy('loan_status').count()
value_rate = value_counts.count() / df_spark.count()
print(f"Number of distincts values: {value_counts.count()} - {value_rate:.2e} %")
value_counts.show()

In [ ]:
df_spark.dtypes

# Data Reduction

With the lack of time, and the complex structure of the table data, mixing types and nan, an advanced analysis would be interesting to pursue a modeling process. However, there is a interesting preprocess that consist to reduce the mix data into a low dimensional relevant data. In fact, this is common in large matrix of data where we can decompose it into a lower format which contains a significative explanative information. This is the case in mathematics with matrix decomposition, and an example of compressing files.

Moreover, reducing the data handled in the modeling part could be interesting for daily professional users. Indeed, assuming the model perform well, it could be boring to insert all the data to assess a loan status. And all the data is not always available for daily uses.

Here, a usefull process is to reduce the amount of features to the more relevant one by gradient boosting modeling. It is possible to extract the features importances of a tree model, whether it concerns classification or regression, and pursue our modeling work on the most important features. The question of the threshold is critical. Here I deciced to limit it by the selecting those until the cumulative exceed 90% of the total sum.

In practice, the use of gradient boosting implementation depends on the data structure or on the underlying computed processes. Even if **[XGBoost](https://xgboost.readthedocs.io/en/stable/)** is widely 
used and usually represent the best models in Kaggle, others librairies exist. Here, I decided to use the library **[Catboost](https://catboost.ai/)** for it is inner preprocessing of categorical variables, which is not present in **XGBoost**. Since I have not enough time to encode all categorical features by myself, it is highly interesting. Of course, the evaluation of feature importances of this model will depends on the inner encoding process of **Catboost**, which will be certainly not reproduced in the next step. Additionnaly, I decided to not tune its hyperparameters or run it through several iterations considering the time allowed. The training was perform on small sample of the entire data to simplify this step and not demanding to much on my computer. I use the library **[Shap](https://shap.readthedocs.io/en/latest/index.html)** to get the features importances ; shapely additive explanations values are based on cooperative game theory.

An interesting cross-validation would be the expertise of professionals who have intuitive knowledge. Obvsiously, the machine learning tools bring probably a better analysis. But, when we hesitate between two features, for example if two feature have a high correlation between them and a similar feature importance. My personal experience with physicians leads me to not exclude those opportunity of cross-validation. 

### Note

If there is a issue when you run the script, even if the seed is set and must reproduce results, here is a script to run before removing unused features.

```python
selected_features = ['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'fico_range_high',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'recoveries',
 'last_pymnt_amnt',
 'last_fico_range_high',
 'last_fico_range_low',
 'mths_since_rcnt_il',
 'mo_sin_old_rev_tl_op',
 'last_credit_pull_d', 
 'last_pymnt_d']
```

In [ ]:
df_gb = df_spark.sample(fraction=0.05, seed=1)
print(f"Number of data: {df_gb.count()}")

In [ ]:
df_gb = df_gb.dropna(subset=['loan_status'])
print(f"Number of data: {df_gb.count()}")

In [ ]:
features_gb = df_gb.select(all_features)
features_collected_gb = features_gb.collect()
target_gb = df_gb.select('loan_status')
target_collected_gb = target_gb.collect()

In [ ]:
X_gb = np.array([list(feature) for feature in features_collected_gb])

In [ ]:
y_gb = np.array([feature['loan_status'] for feature in target_collected_gb])

In [ ]:
X_gb.shape

In [ ]:
y_gb.shape

In [ ]:
categorical_features = [feature for (feature, dtype) in df_gb.dtypes if dtype=='string']

In [ ]:
from catboost import Pool, CatBoostClassifier

pool = Pool(data=X_gb, label=y_gb, feature_names=all_features, cat_features=categorical_features)

catboost_model = CatBoostClassifier(iterations=100)
catboost_model.fit(pool)

In [ ]:
import shap
explainer = shap.Explainer(catboost_model)
shap_values = explainer.shap_values(X_gb)

In [ ]:
sum_over_feature = np.sum(np.abs(shap_values), axis=0)
feature_importance = pd.DataFrame(data=sum_over_feature, index=all_features, columns=['feature_importance'])
feature_importance = feature_importance[feature_importance['feature_importance']>0]
feature_importance = feature_importance.sort_values(by='feature_importance', ascending=False)
feature_importance.shape
feature_importance

In [ ]:
feature_importance['cumulative_sum'] = feature_importance['feature_importance'].cumsum()
total_sum = feature_importance['feature_importance'].sum()
feature_importance['rate'] = feature_importance['cumulative_sum'] / total_sum
feature_importance

In [ ]:
threshold = 0.9
selected_features = feature_importance[feature_importance['rate'] < threshold].index
selected_features
len(selected_features)

In [ ]:
features_to_drop = [feature for feature in all_features if feature not in selected_features]
features_to_drop
len(features_to_drop)

In [ ]:
df_spark = df_spark.drop(*features_to_drop)

# Data Cleaning - Encoding

Now that 17 features are selected, it is time to clean them in a more appropriate way. To analyse the spark dataframe, the code below allows to highlight the cleaning need of each feature. I recommand you to run it before and after the following cleaning scripts if you would like to better understand it.

### Imputation

In a advanced version, I would impute nan with a more sophisticated process than by the mean of each feature. This could be executed by a modeling imputation with the use of machine learning models. Among many possibilities, it exists k-nearest neighbors imputations or deep learning-based imputations. Nevertheless, we can observe than there is only few data containing nan so it is not so critical for a quick hands-on.

In [ ]:
# from pyspark.sql.functions import isnan
# for feature, dtype in df_spark.dtypes:
#     print("====================================")
#     print(f"FEATURE: {feature}")
#     if dtype=='string':
#         value_counts = df_spark.groupBy(feature).count()
#         value_rate = value_counts.count() / df_spark.count()
#         print(f"Number of distincts values: {value_counts.count()} - {value_rate:.2e} %")
#         value_counts.show()
#     nan_count = df_spark.filter(df_spark[feature].isNull() | isnan(df_spark[feature])).count()
#     nan_rate = nan_count / df_spark.count()
#     print(f"{nan_count} NaN - {nan_rate:.2e} %")
#     print("\n")

In [ ]:
print("====================================")
feature = 'last_pymnt_d'
print(f"FEATURE: {feature}")
value_counts = df_spark.groupBy(feature).count()
value_counts.show()

In [ ]:
def to_float(value):
    try:
        return float(value)
    except ValueError:
        return np.nan

from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

to_float_udf = udf(to_float, FloatType())
df_spark = df_spark.withColumn("last_fico_range_high", to_float_udf(df_spark["last_fico_range_high"]))
df_spark = df_spark.withColumn("last_pymnt_amnt", to_float_udf(df_spark["last_pymnt_amnt"]))

In [ ]:
df_spark.createOrReplaceTempView("lending_club_1")
sql_expression = """
CASE
    WHEN last_pymnt_d LIKE 'Jan-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 0/12
    WHEN last_pymnt_d LIKE 'Feb-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 1/12
    WHEN last_pymnt_d LIKE 'Mar-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 2/12
    WHEN last_pymnt_d LIKE 'Apr-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 3/12
    WHEN last_pymnt_d LIKE 'May-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 4/12
    WHEN last_pymnt_d LIKE 'Jun-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 5/12
    WHEN last_pymnt_d LIKE 'Jul-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 6/12
    WHEN last_pymnt_d LIKE 'Aug-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 7/12
    WHEN last_pymnt_d LIKE 'Sep-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 8/12
    WHEN last_pymnt_d LIKE 'Oct-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 9/12
    WHEN last_pymnt_d LIKE 'Nov-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 10/12
    WHEN last_pymnt_d LIKE 'Dec-%' THEN CAST(SUBSTRING(last_pymnt_d, 5) AS FLOAT) + 11/12
    ELSE NULL
END AS last_pymnt_d_num
"""
df_spark = spark.sql(f"""
SELECT *, {sql_expression}
FROM lending_club_1
""")

df_spark = df_spark.drop("last_pymnt_d")
df_spark = df_spark.withColumnRenamed('last_pymnt_d_num', 'last_pymnt_d')

In [ ]:
df_spark.createOrReplaceTempView("lending_club_2")
sql_expression = """
CASE
    WHEN last_credit_pull_d LIKE 'Jan-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 0/12
    WHEN last_credit_pull_d LIKE 'Feb-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 1/12
    WHEN last_credit_pull_d LIKE 'Mar-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 2/12
    WHEN last_credit_pull_d LIKE 'Apr-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 3/12
    WHEN last_credit_pull_d LIKE 'May-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 4/12
    WHEN last_credit_pull_d LIKE 'Jun-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 5/12
    WHEN last_credit_pull_d LIKE 'Jul-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 6/12
    WHEN last_credit_pull_d LIKE 'Aug-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 7/12
    WHEN last_credit_pull_d LIKE 'Sep-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 8/12
    WHEN last_credit_pull_d LIKE 'Oct-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 9/12
    WHEN last_credit_pull_d LIKE 'Nov-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 10/12
    WHEN last_credit_pull_d LIKE 'Dec-%' THEN CAST(SUBSTRING(last_credit_pull_d, 5) AS FLOAT) + 11/12
    ELSE NULL
END AS last_credit_pull_d_num
"""
df_spark = spark.sql(f"""
SELECT *, {sql_expression}
FROM lending_club_2
""")

df_spark = df_spark.drop("last_credit_pull_d")
df_spark = df_spark.withColumnRenamed('last_credit_pull_d_num', 'last_credit_pull_d')

In [ ]:
convert_to_float_rate = udf(lambda x: float(x.replace('%', '')) / 100, FloatType())
df_spark = df_spark.withColumn('int_rate', convert_to_float_rate(df_spark['int_rate']))

In [ ]:
from pyspark.ml.feature import Imputer

numerical_selected_features = [feature for (feature, dtype) in df_spark.dtypes if (dtype!='string' and feature !='loan_status')]

imputer = Imputer(
    inputCols=numerical_selected_features,
    outputCols=numerical_selected_features
).setStrategy("mean")

df_spark = imputer.fit(df_spark).transform(df_spark)

# Data Analysis

### Correlation

It is obvioulsy interesting to analysis correlation, or even partial correlation, between features to identify whether some selected features could be additionnaly removed. Indeed, there could be an issue while modeling colinear features, which could be observed by a brute correlation equals to 1.

Moreover, it is interesting to evaluate the partial correlations by removing the covariance on the different features. It allows to extract the direct relationship between each pair of features. A highly partial correlation could be the origin of an unnecessary dimension modeling. Therefore, the data reduction could be also improved here by removing features included in pair of partial correlation with high values, and which have the lowest features importances, i.e. provided here by gradient boosting. The question of the threshold is still critical here, as it was in the data reduction. 


### Distribution over the *loan_status* target

An other aspect of unrelevant features could be the significative difference between the distribution of features among all the modalities of the target *loan_status*. Here I plot the boxplot of each feature whether it concerns the *Charged Off* or *Fully Paid*, and I run a statistical test to evaluate the null hypothesis that there is a significative difference observing the means of the distinct distribtutions. Sometimes, graphical displays do not match with the statistical test.

### Note

I have honestyly prioritized the modeling section to return a productive work. This section was therefore performed at the end of the hands-on. So the conclusion it provides will not be applied in the rest of this work. Considering the observation of this section, it would be interesting to evaluate the relevance of considering or not each debatable features in the modeling part. 

But keep in mind this analysis was processed on a small set from the entire data...

In [ ]:
df = df_spark.sample(fraction=0.005, seed=1)
print(f"Number of data: {df.count()}")
df_labeled = df.dropna(subset=['loan_status'])
print(f"Number of labeled data: {df_labeled.count()}")

In [ ]:
features = [feature for feature in list(df_spark.columns) if feature != 'loan_status']

In [ ]:
features_collected = df_labeled.select(features).collect()
target_collected = df_labeled.select('loan_status').collect()
X_labeled = np.array([list(feature) for feature in features_collected], dtype=float)
y_labeled = np.array([feature['loan_status'] for feature in target_collected], dtype=float)
y_labeled = y_labeled.astype(int)

In [ ]:
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_combined = pd.DataFrame(X_labeled.astype(float), columns=features)
df_combined['loan_status'] = y_labeled

In [ ]:
num_cols = 5
num_rows = (len(features) + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 4))
axes = axes.flatten()
for i, feature in enumerate(df_combined.columns):
    sns.boxplot(df_combined[feature], ax=axes[i])
    axes[i].set_title(feature)

In [ ]:
plt.figure(figsize=(10, 6))
sns.pairplot(df_combined[features])
plt.show()

In [ ]:
corr_matrix = df_combined.corr(numeric_only=False).round(2)
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, cbar_kws={'label': 'Partial Correlation'})
plt.title('Correlation Heatmap')
plt.show()

pcorr_matrix = df_combined.pcorr().round(2)
plt.figure(figsize=(10, 8))
sns.heatmap(pcorr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, cbar_kws={'label': 'Partial Correlation'})
plt.title('Partial Correlation Heatmap')
plt.show()

In [ ]:
num_cols = 6
num_rows = 6
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 4))
axes = axes.flatten()
for i, feature in enumerate(features):
    df_fully_paid = df_combined[df_combined['loan_status'] == 0.0][feature]
    df_charged_off = df_combined[df_combined['loan_status'] == 1.0][feature]
    sns.boxplot(df_fully_paid.tolist(), ax=axes[2*i], color='C1')
    axes[2*i].set_title(f"Fully Paid\n{feature}")
    sns.boxplot(df_charged_off.tolist(), ax=axes[2*i+1], color='C2')
    axes[2*i+1].set_title(f"Charged Off\n{feature}")

In [ ]:
from scipy.stats import ttest_ind, f_oneway

fully_paid = df_combined[df_combined['loan_status'] == 0][features]
charged_off = df_combined[df_combined['loan_status'] == 1][features]
t_stat, p_val = ttest_ind(fully_paid, charged_off)

for i, feature in enumerate(features):
    print(f"Feature: {feature}")
    print(f"\tT-statistic: {t_stat[i]}")
    print(f"\tP-value: {p_val[i]}")
    if p_val[i] < 0.05:
        print("\tReject the null hyptothesis ~ Significant difference between the means of binary 'loan_status'")
    else:
        print("\tFail to reject the null hyptothesis ~ No significant difference between the means of binary 'loan_status'")
    print("\n")
    

In [ ]:
from pyspark.sql.functions import isnan
df_nan = df.filter(isnan(df['loan_status']))
print(f"Number of NaN data: {df_nan.count()}")

In [ ]:
features_collected = df_nan.select(features).collect()
X_unlabeled = np.array([list(feature) for feature in features_collected], dtype=float)

In [ ]:
classes_ = np.unique(y_labeled)

# Modeling

When it comes to choose a machine learnning classifier to predict *loan_status*, two main classes are well known to perform good evaluation: deep neural networks and gradient boosting. *XGBoost* is one of the best model in common Kaggle competition. Several articles compare their performances:

**Article**: [Why do tree-based models still outperform deep learning on typical tabular data?](https://proceedings.neurips.cc/paper_files/paper/2022/hash/0378c7692da36807bdec87ab043cdadc-Abstract-Datasets_and_Benchmarks.html)

**Article**: [Credit Risk Assessment based on Gradient Boosting Decision Tree](https://www.sciencedirect.com/science/article/pii/S1877050920315842)

**Article**: [Deep Learning vs. Gradient Boosting: Benchmarking state-of-the-art machine learning algorithms for credit scoring](https://arxiv.org/abs/2205.10535)

However, to illustrate my abilities in this hands-on, I choose to apply a deep multilayer perceptron network to classify loans since I already use a gradient boosting model for data reduction. Initially, since the goal of this work is to target only two *loan_status* among a dozen, I compare it to spam filtered in Google. Even if it has evolved, I keep in mind that the underlying process is related to semi-supervised classification. This involves unlabeled and labeled data. 

Additionnaly, even if I have already work on TensorFlow and the high level Keras framework, I choose to apply Pytorch to gain some times since I am habit to work on it in my current schedule. I have added several elements to its model to bring a little more complexity.

### Scalability

The rate of the train set could be filled when initializing the model. Nevertheless, I recommand to use between 30% and 15% of the entire data to keep a reliable test set to evaluate properly the model. Here, the amount of data is too large for a quick hands-on. You can see that I have also test my model on larger sample of the spark dataframe than the training sample, and the evaluation still pretty nice. If I have more time, I would probably try to take a very larger size of data. 

### Conformal predicted intervals

**[Mapie](https://mapie.readthedocs.io/en/latest/)** library provides conformal predicted intervals, whether it concerns regression or classification. However, it could be only applied on **Scikit-learn** library format ; explaining why I must to implement a intermediate class named **MLPEstimatorSklearn**. Unfortunaly, I still have a issue while I deliver this hands-on: while *fordward_proba* effectively provides a float, the inner process of **MapieClassifier** probably translate it into a boolean since float values could be extremely near from both 0 and 1. And I suppose their is a process that translate 0 and 1 to boolean, like in C language. Something like this, I did not deepen this issue.

**Article**: [Conformal Prediction Sets for Ordinal Classification](https://proceedings.neurips.cc/paper_files/paper/2023/hash/029f699912bf3db747fe110948cc6169-Abstract-Conference.html)

**Article**: [Conjugate Conformal Prediction for Online Binary Classification](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=77fbaec8faa58fc547de29082538b16c328545df)

### Hyperparameters tuning

Inner hyperparameters tuning whether it is by calling randomized or bayes optimization. I fill a range for each hyperparameter which could be debatable. I have set string in hidden_layer_size hyperparameter which represent the hidden architecture of the MLP classifier since bayes search could not accept a tuple of tuple for its range, even with extra functions from **Skopt** such as *Optional*. This is why I handle it through tuple in the low-level implementation of **MLP**. Unfornutalely, I also have an issue with bayes search. While randomized search work, bayesian tuning seems to be not up to date to **Numpy** library. And I was not able to find a quick solution within the time allowed. If you want to optimize hyperparamaters of the model, please run the method *randomized_search* before fitting the model. Note also that it could not explore the initial hyperparameters settings, which is already highly interesting. Nevertheless, it is also highly possible to find a better architecture, for example, with enough iterations of search. Even if the exploration is limited here, as a reminder, there is not a suitable architecture for any problems since there is not prior strategy to find the optimal architecture.

### Interprating method through integrated gradients

Deep neural networks are advanced black-box processes which are difficult to interpret. However, it exists some methods to bring interpration on their well performance. One of the most interesting one is integrated gradients since it checks many axioms. By interpolating gradients, it is possible to identify which features is the most important for the attribution of a deep neural netwoks, whether it concerns classification or regression for example, even in NLP, Computer Vision fields. Here I use **[Captum](https://captum.ai/)** libraby which is linked to **Pytorch** to accelerate its computing. It involves NVIDIA libraries to parallel computing, explaining the quite large size of the docker image of my trained MLP binary classifier. I do not recommend to run *compute_integreated_gradients*  on a large amount to not freeze your computer, which was my case. Nevertheless, it allows to provides the importances of each feature toward the attribution of the *loan_status* attribution. I decided to normalized it in min max standardization to better catch the importances, while the original value is extremely small.

**Article** : [Axiomatic Attribution for Deep Networks](http://proceedings.mlr.press/v70/sundararajan17a.html)


However, Kolmogorov Arnold networks are a well alternative to black-box neural networks. Without going deepen to desrible the KAN models since it is not the subject of its hands-on, it computes activation functions by the use of splines, instead of computing weights and biaises. It could be very interesting if you would like to get a "generic" function, linking intputs to outputs. It generally needs a smaller architecture to provides same evaluation than a black-box neural networks. A main interest coulb be explain when physician would like to identify a generic function of a unknown process, for example.


### Note

I have tried different strategies to check the performance of the model. Even if I know it is better to target a single neuron in the output layer in case of binary classifier, I also tried to used two neurons. Why I did this? Because I hesitated on the well approach to return probabilities of the two class. I know that softmax activation function is the activation function to apply to return probabilities for each neuron in a multi-class classifier. While sigmoid function also returns value between 0 and 1, I honestly hesitated whether or not is it appropriate to return the probability of the class 1, i.e. *Charged Off* here. 

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from captum.attr import IntegratedGradients
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from mapie.classification import MapieClassifier
from sklearn.calibration import CalibratedClassifierCV
from skopt import BayesSearchCV
import matplotlib.pyplot as plt

torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layer_sizes, activation_name, p_dropout):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layer_sizes = hidden_layer_sizes
        if isinstance(self.hidden_layer_sizes, str):
            self.hidden_layer_sizes = eval(self.hidden_layer_sizes)
        self.activation_name = activation_name
        self.p_dropout = p_dropout
        if activation_name == "Relu":
            self.activation = nn.ReLU()
        elif activation_name == "Sigmoid":
            self.activation = nn.Sigmoid()
        elif activation_name == "Softmax":
            self.activation = nn.Softmax(dim=1)
        elif activation_name == "Tanh":
            self.activation = nn.Tanh()
        elif activation_name == "Leaky_relu":
            self.activation = nn.LeakyReLU()
        else:
            raise ValueError(f'Unsupported activation: {self.activation_name}')
        layers = []
        layers.append(nn.Linear(self.input_size, self.hidden_layer_sizes[0]))
        layers.append(self.activation)
        for i in range(len(self.hidden_layer_sizes) - 1):
            layers.append(nn.Linear(self.hidden_layer_sizes[i], self.hidden_layer_sizes[i + 1]))
            layers.append(self.activation)
            layers.append(nn.Dropout(p=self.p_dropout))
        layers.append(nn.Linear(self.hidden_layer_sizes[-1], self.output_size))
        self.model = nn.Sequential(*layers)
        self.sigmoid = nn.Sigmoid()
        

    def forward_proba(self, X):
        output = self.model(X)
        output = self.sigmoid(output)
        return output.float()
    
    
    def forward(self, X):
        output = self.model(X)
        output = self.sigmoid(output)
        return output


class MLPEstimatorSklearn():
    def __init__(self, **params):
        self.input_size = params.get("input_size")
        self.output_size = params.get("output_size")
        self.hidden_layer_sizes = params.get("hidden_layer_sizes", (60, 60))
        self.activation_name = params.get("activation_name", "Relu")
        self.loss = params.get("loss", "binary_cross_entropy")
        self.optimizer_name = params.get("optimizer_name", "Adam")
        self.learning_rate = params.get("learning_rate", 1e-3)
        self.batch_size = params.get("batch_size", 50)
        self.weight_decay = params.get("weight_decay", 0)
        self.p_dropout = params.get("p_dropout", 0.2)
        self.early_stopping = params.get("early_stopping", True)
        self.epochs = params.get("epochs", 200)
        self.patience = params.get("patience", 10)
        self.verbose = params.get("verbose", True)
        self.classes_ = classes_

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = MLP(self.input_size, self.output_size, self.hidden_layer_sizes, self.activation_name, self.p_dropout).to(self.device)

        if self.loss == "binary_cross_entropy":
            self.criterion = nn.BCELoss()
        else:
            raise ValueError(f"Unsupported loss: {self.loss}")

        if self.optimizer_name == "SGD":
            self.optimizer = optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=0.9, weight_decay=self.weight_decay)
        elif self.optimizer_name == "Adam":
            self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer}")
            
    def next_batch(self, inputs, targets, batchSize):
        inputs_tensor = torch.from_numpy(inputs).float()
        targets_tensor = torch.from_numpy(targets).float().unsqueeze(1)
        for i in range(0, inputs_tensor.shape[0], batchSize):
            yield (inputs_tensor[i:i + batchSize], targets_tensor[i:i + batchSize])

    def augment_data(self, X_unlabeled, noise_level=0.1):
        noise = noise_level * torch.randn_like(X_unlabeled)
        return X_unlabeled + noise
            
    def fit(self, X, y, X_unlabeled=None):
        self.classes_ = np.unique(y)
        running_losses_1 = list()
        if self.early_stopping:
            best_loss = float('inf')
            count = 0
        if self.verbose:
            epoch_iterator_1 = tqdm(range(self.epochs), desc="Supervised training ; epochs", unit="epoch")
        else:
            epoch_iterator_1 = range(self.epochs)
        for epoch in epoch_iterator_1:
            samples = 0
            train_loss = 0.0
            self.model.train(True)
            for i, (batchX, batchY) in enumerate(self.next_batch(X, y, self.batch_size)):
                batchX = batchX.to(self.device)
                batchY = batchY.to(self.device)
                batchY.requires_grad = True
                self.optimizer.zero_grad()
                outputs = self.model(batchX)
                loss = self.criterion(outputs, batchY)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
                samples += batchY.size(0)
            running_loss = train_loss / samples
            running_losses_1.append(running_loss)
            if self.verbose:
                epoch_iterator_1.set_postfix(train_loss=running_loss)
            if self.early_stopping:
                if running_loss < best_loss:
                    best_loss = running_loss
                    count = 0
                else:
                    count += 1
                if count >= self.patience:
                    break
        if self.verbose:
            epoch_iterator_1.close()
        self.running_losses_1 = [loss for loss in running_losses_1 if loss <= best_loss]
        if X_unlabeled is not None:
            best_loss = float('inf')
            running_losses_2 = list()
            X_unlabeled = torch.from_numpy(X_unlabeled).float()
            augmented_X_unlabeled = self.augment_data(X_unlabeled)
            self.model.eval()
            with torch.no_grad():
                pseudo_labels = self.model(X_unlabeled)
                pseudo_labels = (pseudo_labels > 0.5).float().squeeze()
            
            X_combined = torch.cat((torch.from_numpy(X).float(), augmented_X_unlabeled.cpu()), 0).to(self.device)
            y_combined = torch.cat((torch.from_numpy(y).float().squeeze(), pseudo_labels), 0).to(self.device)

            if self.verbose:
                epoch_iterator_2 = tqdm(range(self.epochs), desc="Semi-supervised training ; epochs", unit="epoch")
            else:
                epoch_iterator_2 = range(self.epochs)
            for epoch in epoch_iterator_2:
                samples = 0
                train_loss = 0.0
                self.model.train(True)
                dataset = TensorDataset(X_combined, y_combined)
                for i, (batchX, batchY) in enumerate(self.next_batch(X, y, self.batch_size)):
                    batchX = batchX.to(self.device)
                    batchY = batchY.to(self.device)
                    batchY.requires_grad = True
                    self.optimizer.zero_grad()
                    outputs = self.model(batchX)
                    loss = self.criterion(outputs, batchY)
                    loss.backward()
                    self.optimizer.step()
                    train_loss += loss.item()
                    samples += batchY.size(0)
                running_loss = train_loss / samples
                running_losses_2.append(running_loss)
                if self.verbose:
                    epoch_iterator_2.set_postfix(train_loss=running_loss)
                if self.early_stopping:
                    if running_loss < best_loss:
                        best_loss = running_loss
                        count = 0
                    else:
                        count += 1
                    if count >= self.patience:
                        break
            if self.verbose:
                epoch_iterator_2.close()
            self.running_losses_2 = [loss for loss in running_losses_2 if loss <= best_loss]
        return self

    def predict(self, X):
        self.model.eval()
        X = torch.from_numpy(X).float().to(self.device)
        y_pred = self.model.forward(X)
        if self.device == "cpu":
            y_pred = y_pred.cpu().detach().numpy()
        else:
            y_pred = y_pred.detach().numpy()
        return (y_pred > 0.5).astype(int)

    def predict_proba(self, X):
        self.model.eval()
        X = torch.from_numpy(X).float().to(self.device)
        y_proba = self.model.forward_proba(X)
        if self.device == "cpu":
            y_proba = y_proba.cpu().detach().numpy().astype(float)
        else:
            y_proba = y_proba.detach().numpy().astype(float)
        y_proba = y_proba.squeeze().astype(np.float32)
        return np.column_stack((1 - y_proba, y_proba))

    def get_params(self, deep=True):
        params = {
            "input_size": self.input_size,
            "output_size": self.output_size,
            "hidden_layer_sizes": self.hidden_layer_sizes,
            "activation_name": self.activation_name,
            "loss": self.loss,
            "optimizer_name": self.optimizer_name,
            "learning_rate": self.learning_rate,
            "batch_size": self.batch_size,
            "weight_decay": self.weight_decay,
            "p_dropout": self.p_dropout,
            "early_stopping": self.early_stopping,
            "epochs": self.epochs,
            "patience": self.patience,
            "verbose": self.verbose
        }
        return params

    def set_params(self, **params):
        for key, value in params.items():
            if hasattr(self, key):
                setattr(self, key, value)
        return self

    def get_mlp(self):
        return self.model
    
class MLPBinaryClassifier():
    def __init__(self, X, y, split_test, X_unlabeled=None, **params):
        self.model = MLPEstimatorSklearn(**params)
        self.X = X
        self.X_unlabeled = X_unlabeled
        self.y = y
        
        self.y = MLPBinaryClassifier.float_to_class(self.y).ravel()
        
        self.split_test = split_test
        self.split_data()
        
        self.standardize(self.X_train_cal)
        self.X_train_standard = self.standardize_X(self.X_train)
        self.X_cal_standard = self.standardize_X(self.X_cal)
        if isinstance(self.X_unlabeled, np.ndarray):
            self.X_unlabeled_standard = self.standardize_X(self.X_unlabeled)
        else :
            self.X_unlabeled_standard = None
        self.y_train_standard = self.y_train
        self.y_cal_standard = self.y_cal.reshape(-1,1)


    @staticmethod
    def float_to_class(y):
        threshold = 0.5
        return (y >= threshold).astype(int)
    
    def split_data(self):
        self.X_train_cal, self.X_test, self.y_train_cal, self.y_test = train_test_split(
            self.X, self.y, test_size=self.split_test, shuffle=True, random_state=1, stratify=self.y)
        self.X_train, self.X_cal, self.y_train, self.y_cal = train_test_split(
            self.X_train_cal, self.y_train_cal, test_size=0.25, shuffle=True, random_state=1, stratify=self.y_train_cal)

    def standardize(self, X):
        self.scaler_X_train = StandardScaler()
        self.scaler_X_train.fit(X)         


    def standardize_X(self, X):
        X_new = self.scaler_X_train.transform(X)
        return X_new
    


    def bayes_search(self, param_bayes, n_iter, n_points=1, cv=5, scoring='accuracy',
                 verbose=3, n_jobs=1) :
        cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=1)
        bayes_search = BayesSearchCV(self.model, param_bayes, n_iter=n_iter,
                                     n_points=n_points, cv=cv, scoring=scoring,
                                     verbose=verbose, return_train_score=True,
                                     n_jobs=n_jobs, random_state=1)
        bayes_search.fit(self.X_train_standard, self.y_train_standard)
        results_df = pd.DataFrame(bayes_search.cv_results_)
        self.model = bayes_search.best_estimator_
        print(f'Best hyperparameters bayes search : {bayes_search.best_params_}')
        return results_df

    def randomized_search(self, param_randomized, n_iter, cv=5, scoring='accuracy',
                      verbose=3, n_jobs=1) :
        cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=1)
        randomized_search = RandomizedSearchCV(self.model, param_randomized,
                                               n_iter=n_iter, cv=cv, scoring=scoring,
                                               verbose=verbose, return_train_score=True,
                                               n_jobs=n_jobs, random_state=1)
        randomized_search.fit(self.X_train_standard, self.y_train_standard)
        results_df = pd.DataFrame(randomized_search.cv_results_)
        self.model = randomized_search.best_estimator_
        print(f'Best hyperparameters randomized search : {randomized_search.best_params_}')
        return results_df

    def fit(self, method="lac"):
        self.model.fit(self.X_train_standard, self.y_train_standard, self.X_unlabeled_standard)
        self.model_mapie = MapieClassifier(estimator=self.model, cv="prefit", method=method)
        self.model_mapie.fit(self.X_cal_standard, self.y_cal_standard)

    def predict(self, X, alpha=0.05):
        X_standard = self.standardize_X(X)
        y_pred, y_ps = self.model_mapie.predict(X_standard, alpha=alpha)
        return y_pred, y_ps

    @staticmethod
    def compute_metrics(metric, y_true, y_pred):
        y_pred = MLPBinaryClassifier.float_to_class(y_pred)
        accuracy = metrics.accuracy_score(y_true, y_pred)
        precision = metrics.precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = metrics.recall_score(y_true, y_pred, average='weighted')
        f1 = metrics.f1_score(y_true, y_pred, average='weighted')
        metrics_dict = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
        }
        if metric != 'all':
            metrics_dict = {metric: metrics_dict[metric]}
        return metrics_dict


    def model_performance(self, metric='all'):
        y_pred_train, _ = self.predict(self.X_train)
        scores_train = MLPBinaryClassifier.compute_metrics(metric, self.y_train, y_pred_train)
        y_pred_test, _ = self.predict(self.X_test)
        scores_test = MLPBinaryClassifier.compute_metrics(metric, self.y_test, y_pred_test)
        data = {}
        for key, value in scores_train.items():
            data['Train Set - '+key] = [value]
        for key, value in scores_test.items():
            data['Test Set - '+key] = [value]
        df_scores = pd.DataFrame(data=data).T
        df_scores.columns = ['Scores']
        return df_scores

    def model_performance_test(self, X_test, y_test, metric='all'):
        y_pred_test, _ = self.predict(X_test)
        scores_test = MLPBinaryClassifier.compute_metrics(metric, y_test, y_pred_test)
        data = {}
        for key, value in scores_test.items():
            data['Test Set - '+key] = [value]
        df_scores = pd.DataFrame(data=data).T
        df_scores.columns = ['Scores']
        return df_scores

    def receiver_operating_characteristics(self):
        y_pred_test, _ = self.predict(self.X_test)
        fpr, tpr, thresholds = metrics.roc_curve(self.y_test, y_pred_test)
        plt.plot(fpr, tpr)
        plt.title("Receiver Operating Characteristics")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.show()

    def compute_integrated_gradients(self, X, baseline=None, steps=50):
        def preprocess_input(X):
            return torch.tensor(X, dtype=torch.float32)
        input_tensor = preprocess_input(X)
        if baseline is None:
            baseline = torch.zeros_like(input_tensor)
        integrated_gradients = IntegratedGradients(self.model.get_mlp())
        attributions = integrated_gradients.attribute(input_tensor, baseline, target=0, n_steps=steps)
        attributions_df = pd.DataFrame(attributions.cpu().detach().numpy(), columns=features)
        avg_attributions = attributions_df.mean(axis=0)
        avg_abs_attributions = avg_attributions.abs()
        def custom_minmax_scaler(data, feature_range=(0, 100)):
            min_val = np.min(data)
            max_val = np.max(data)
            if max_val - min_val == 0:
                return np.zeros_like(data) if feature_range[0] == 0 else np.full_like(data, feature_range[0])
            scale = (feature_range[1] - feature_range[0]) / (max_val - min_val)
            min_range = feature_range[0]
            scaled_data = scale * (data - min_val) + min_range
            return scaled_data
        normalized_data = custom_minmax_scaler(avg_abs_attributions.values.reshape(-1, 1)).astype(float)
        np.set_printoptions(suppress=True, precision=2)
        normalized_attributions = pd.DataFrame(normalized_data, columns=['attribution'], index=features)
        sorted_attributions = normalized_attributions.sort_values(by="attribution", ascending=False)
        return sorted_attributions

In [ ]:
from scipy.stats import loguniform, uniform
from skopt.space import Real

params = {    
    "init" : {
        "input_size" : len(features),
        "output_size" : 1,
        "hidden_layer_sizes" : (60,60),
        "activation_name" : "Relu",
        "optimizer_name" : "Adam",
        "learning_rate" : 1e-3,
        "batch_size" : 50,
        "weight_decay" : 0,
        "p_dropout" : 0.3,
        "loss" : "binary_cross_entropy",
        "early_stopping" : True,
        "epochs" : 200,
        "patience" : 10,
        "verbose" : True
    },
    "randomized": {
        "hidden_layer_sizes" : [(10,),(50,),(100,),(10,10),(50,50),(60,60),(100,50),(100,100),(100,50,25)],
        "activation_name" :  ["Relu", "Sigmoid", "Tanh", "Leaky_relu", "Softmax"],
        "learning_rate" : loguniform(1e-4, 1e-1),
        "batch_size" : list(np.arange(10,500, 10)),
        "optimizer_name" : ["Adam", "SGD"],
        "alpha" : np.logspace(-3,0,19),
        "weight_decay" : loguniform(1e-5, 1),
        "p_dropout" : uniform(0, 0.4)   
    },
    "bayes": {
        "hidden_layer_sizes" : ["(10,)","(50,)","(100,)","(10,10)","(50,50)","(60,60)","(100,50)","(100,100)","(100,50,25)"],
        "activation_name" :  ["Relu", "Sigmoid", "Tanh", "Leaky_relu", "Softmax"],
        "learning_rate" : Real(1e-4, 1e-1, prior='log-uniform'),
        "batch_size" : list(np.arange(10,500, 10)),
        "optimizer_name" : ["Adam", "SGD"],
        "alpha" : np.logspace(-3,0,19),
        "weight_decay" : Real(1e-5, 1, prior='log-uniform'),
        "p_dropout" : Real(0, 0.4, prior='uniform')
    }
}

model_mlp = MLPBinaryClassifier(X=X_labeled, y=y_labeled, X_unlabeled=X_unlabeled, split_test=0.2, **params["init"])

In [ ]:
n_iter = 5
n_points=1
cv=5
scoring='accuracy'
verbose=3
n_jobs=-1

In [ ]:
# model_mlp.bayes_search(
#     param_bayes=params['bayes'],
#     n_iter=n_iter,
#     n_points=n_points,
#     cv=cv,
#     scoring=scoring,
#     n_jobs=n_jobs
# )

In [ ]:
# model_mlp.randomized_search(
#     param_randomized=params['randomized'],
#     n_iter=n_iter,
#     cv=cv,
#     scoring=scoring,
#     n_jobs=n_jobs
# )

In [ ]:
model_mlp.model.get_params()

In [ ]:
model_mlp.fit()

# Evaluation



For a classification problem, few elements must be identified:

- **TP** (True Positives) is the number of correctly classified *Charged Off* loans.
- **TN** (True Negatives) is the number of correctly classified *Fully Paid* loans.
- **FP** (False Positives) is the number of *Fully Paid* loans incorrectly classified as 'Charged Off'.
- **FN** (False Negatives) is the number of *Charged Off* loans incorrectly classified as 'Fully Paid'.

Here a metrics to evaluate the performance of the model:

*Precision, Recall and F1Score provide a better understanding of how well the model is performing in identifying the *Charge Off* loans, the minority target of the dataset.*

### Accuracy

This metrics is the main metric to evaluate the model through the hyperparameters search.

$$ Accuracy = \frac{TP + TN}{TP + TN + FP + FN} $$

### Precision

$$ Precision = \frac{TP}{TP + FP} $$

### Recall - Sensitivity

$$ Accuracy = \frac{TP}{TP + FN} $$

### F1Score

$$ F1Score = 2 * \frac{Precision * Recall}{Precision + Recall} $$

### ROC-AUC

The area under the receiver operation characteristic curve is a single value summarizing the overall ability of the model.

### Observations

The trained model well performed a binary classification on the dataset since metrics are near from 1, i.e. higher than 0,99.

In [ ]:
model_mlp.model_performance()

In [ ]:
model_mlp.receiver_operating_characteristics()

In [ ]:
df_test = df_spark.sample(fraction=0.05, seed=30)
df_test = df_test.dropna(subset=['loan_status'])
features_collected = df_test.select(features).collect()
X_test = np.array([list(feature) for feature in features_collected])
target_collected = df_test.select('loan_status').collect()
y_test = np.array([feature['loan_status'] for feature in target_collected])
y_test = y_test.astype(int)

In [ ]:
model_mlp.model_performance_test(X_test, y_test)

In [ ]:
single_value_collected = df_spark.sample(withReplacement=False, fraction=0.0001, seed=1).limit(1).collect()[0]
single_value = np.array([value for key, value in single_value_collected.asDict().items() if key != 'loan_status']).reshape(1,-1).astype(float)
single_value_target = np.array([value for key, value in single_value_collected.asDict().items() if key == 'loan_status'])

In [ ]:
model_mlp.compute_integrated_gradients(single_value)

# Data Visualization

If I have more time, I would like to display the prediction areas of the classifier in multiple 2D or 3D graphics, taking different features in axes.

# Saving 

I saved a trained MLP classifier model with initial configuration for my docker image.

In [ ]:
import pickle
with open('app/lending_club_mlp_binary_classifier.pkl', 'wb') as file:
    pickle.dump(model_mlp, file)

# Application

The docker image of the application is located [here](https://hub.docker.com/repository/docker/yanncauchepin/lendingclub/general).

I realized that I have not handle the case where one of the feature filled to describe the data is NaN. I would probably code a inner function of the model to process an imputation with a more advanced method than just setting the mean value of the feature.

Due to an additionnal line that I forgot to removed in the past, the application and its model do not handle the feature *last_pymnt_d* while it is well selected.

# Thank you

I would like to thank you for taking the time to read this hands-on! 